# Lab3

### 環境建置

In [11]:
# TEST necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Input, BatchNormalization, LSTM, GRU, Bidirectional
from tensorflow.keras.layers import TimeDistributed, Reshape, Concatenate
from tensorflow.keras.regularizers import l2

from tensorflow import keras

from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


### 模型

In [13]:

def build_model():
    # 使用固定的參數
    conv_filters = 64
    dropout_rate = 0.2
    lstm_units = 96
    learning_rate = 0.0005
    
    # 建立模型
    inputs = keras.Input(shape=input_shape)
    
    # CNN部分
    x = layers.Conv2D(filters=conv_filters, kernel_size=(1, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(1, 2))(x)
    x = layers.Conv2D(filters=conv_filters*2, kernel_size=(1, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(1, 2))(x)
    
    # 重塑以便LSTM處理
    x = layers.TimeDistributed(layers.Flatten())(x)
    
    # 雙向LSTM部分
    x = layers.Bidirectional(layers.LSTM(units=lstm_units, return_sequences=True))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Bidirectional(layers.LSTM(units=lstm_units//2))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # 全連接層
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # 編譯模型
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

### 資料讀取

In [14]:


# 讀取數據
#df = pd.read_csv("../WESAD/s2_data.csv")
df = pd.read_csv("../Output/lab3_data.csv")
df = df.sample(frac=0.01)
print(f"數據集大小: {len(df)}")

# 分開特徵和標籤
X = df.drop(columns=['Label','Subject'])  # 移除標籤 (y)  
y = df['Label'] 
print(f"特徵矩陣形狀: {X.shape}")
print("特徵數據前5行:")
print(X.head())

# 確定實際特徵數量
n_features = X.shape[1]
print(f"特徵數量: {n_features}")

num_classes = 2
print(f"類別數量: {num_classes}")

# 正確的數據重塑方式 - 根據特徵結構調整
X_reshaped = X.values.reshape(X.shape[0], 1, n_features, 1)
input_shape = (1, n_features, 1)
print(f"重塑後的數據形狀: {X_reshaped.shape}")

# 標籤處理 - 將標籤調整為從0開始
y_adjusted = y - 1  # 如果標籤是1和2，調整為0和1

# 切分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_adjusted, test_size=0.2, random_state=42  
)




數據集大小: 193039
特徵矩陣形狀: (193039, 5)
特徵數據前5行:
          ecg_singal  EMG_signal  EDA_signal  Resp_signal  Temp_signal
1472060     0.479088    0.589569    0.046691     0.617289     0.993098
4445638     0.490501    0.588455    0.136264     0.599632     0.993648
190439      0.498848    0.593595    0.259311     0.537945     0.991568
16771873    0.497551    0.592571    0.249958     0.581228     0.993343
7882752     0.515007    0.587179    0.035938     0.602479     0.994480
特徵數量: 5
類別數量: 2
重塑後的數據形狀: (193039, 1, 5, 1)


In [15]:


# 定義早停策略
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 使用固定的參數建立模型
model = build_model()

# 訓練模型
batch_size = 32  # 您也可以調整批次大小
final_history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=100,  # 可以設置較長的訓練時間，因為有早停機制
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# 評估最終模型
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'測試準確率: {test_acc:.4f}')



Epoch 1/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.6942 - loss: 0.5776 - val_accuracy: 0.7563 - val_loss: 0.4908
Epoch 2/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.7748 - loss: 0.4730 - val_accuracy: 0.7886 - val_loss: 0.4587
Epoch 3/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.7827 - loss: 0.4577 - val_accuracy: 0.7924 - val_loss: 0.4325
Epoch 4/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.7894 - loss: 0.4464 - val_accuracy: 0.7927 - val_loss: 0.4259
Epoch 5/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8050 - loss: 0.4178 - val_accuracy: 0.8406 - val_loss: 0.3899
Epoch 6/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8244 - loss: 0.4011 - val_accuracy: 0.8443 - val_loss: 0.3736
Epoch 7/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8287 - loss: 0.3934 - val_accuracy: 0.8446 - val_loss: 0.3666
Epoch 8/100
4826/4826 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8389 - loss: 

### 訓練

In [16]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# 預測測試集
y_pred = model.predict(X_test)

# 將預測的機率轉換為類別標籤
y_pred_labels = np.argmax(y_pred, axis=1)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"模型準確率: {accuracy:.4f}")

# 輸出分類報告
print("分類報告：")
print(classification_report(y_test, y_pred_labels, digits=4))


1207/1207 ━━━━━━━━━━━━━━━━━━━━ 2s 971us/step
模型準確率: 0.9055
分類報告：
              precision    recall  f1-score   support

         0.0     0.9119    0.9426    0.9270     24585
         1.0     0.8931    0.8403    0.8659     14023

    accuracy                         0.9055     38608
   macro avg     0.9025    0.8915    0.8964     38608
weighted avg     0.9051    0.9055    0.9048     38608

